This notebook is to inspect the touch signals and make them compatible as model input.

In [43]:
import pandas as pd
import itertools

 -- 1. Read your data --

In [44]:
# Define the file path
touch_path = "/Users/joshualunger/DeepEmotion/data/annotations/src/bodycontact/data/obs4.csv"
emotion_path = "/Users/joshualunger/DeepEmotion/data/resampled_annotations/av1o6_resampled.tsv"
# Read the CSV file
touch_df = pd.read_csv(touch_path)
emotion_df = pd.read_csv(emotion_path, sep='\t') 

# Print the first few rows
print(touch_df.head())
print(emotion_df.head())


    start     end     actor recipient bodypart_actor bodypart_recipient  \
0  281.72  286.20    OLDMAN   FORREST            ARM       BACK ARM LEG   
1  285.76  286.20    OLDMAN   FORREST            ARM              CHEST   
2  289.72  289.84  MRS_GUMP   FORREST           HAND               HAND   
3  290.00  290.92  MRS_GUMP   FORREST           HAND                LEG   
4  314.24  316.32  MRS_GUMP   FORREST           HAND               HAND   

             label  intensity_of_body_contact valence_actor valence_recipient  \
0  CHANGE_POSITION                          1      POSITIVE          POSITIVE   
1              TAP                          0      POSITIVE          POSITIVE   
2       BRUSH PAST                          0      POSITIVE          POSITIVE   
3     FIX_CLOTHING                          0      POSITIVE          POSITIVE   
4       HOLD_HANDS                          0      POSITIVE          POSITIVE   

   intention audio_information  
0          1         NARRATIO

-- 2. Extract unique body parts --

In [45]:
# 1. Convert NaN to empty string to avoid errors when splitting
touch_df['bodypart_actor'] = touch_df['bodypart_actor'].fillna('')
touch_df['bodypart_recipient'] = touch_df['bodypart_recipient'].fillna('')

# 2. Split each cell on whitespace, collect into lists
actor_bodyparts_lists = touch_df['bodypart_actor'].apply(lambda x: x.split())
recipient_bodyparts_lists = touch_df['bodypart_recipient'].apply(lambda x: x.split())

# 3. Flatten these lists and find unique body parts
all_actor_bps = set(itertools.chain.from_iterable(actor_bodyparts_lists))
all_recipient_bps = set(itertools.chain.from_iterable(recipient_bodyparts_lists))

all_unique_bodyparts = sorted(all_actor_bps.union(all_recipient_bps))

actor_columns = [f"{bp}_actor" for bp in all_unique_bodyparts]
recipient_columns = [f"{bp}_recipient" for bp in all_unique_bodyparts]

print(all_unique_bodyparts)
print(actor_columns)
print(recipient_columns)

['ABDOMEN', 'ARM', 'BACK', 'BUTTOCKS', 'CHEST', 'FACE', 'FOOT', 'HAND', 'HEAD', 'HIP', 'LAP', 'LEG', 'NAPE', 'NECK', 'SHOULDER']
['ABDOMEN_actor', 'ARM_actor', 'BACK_actor', 'BUTTOCKS_actor', 'CHEST_actor', 'FACE_actor', 'FOOT_actor', 'HAND_actor', 'HEAD_actor', 'HIP_actor', 'LAP_actor', 'LEG_actor', 'NAPE_actor', 'NECK_actor', 'SHOULDER_actor']
['ABDOMEN_recipient', 'ARM_recipient', 'BACK_recipient', 'BUTTOCKS_recipient', 'CHEST_recipient', 'FACE_recipient', 'FOOT_recipient', 'HAND_recipient', 'HEAD_recipient', 'HIP_recipient', 'LAP_recipient', 'LEG_recipient', 'NAPE_recipient', 'NECK_recipient', 'SHOULDER_recipient']


-- 3. Prepare an empty result list --

In [46]:
aligned_rows = []

for i, emotion_row in emotion_df.iterrows():
    current_offset = emotion_row['offset']

    # -- Find all touch events active at this offset
    active_touches = touch_df[
        (touch_df['start'] <= current_offset) &
        (touch_df['end'] >= current_offset)
    ]

    # -- Initialize body part one-hot dict for both actor & recipient
    bodypart_encoding = {col: 0 for col in actor_columns + recipient_columns}

    # -- For each active touch event, parse multiple body parts
    for _, touch_row in active_touches.iterrows():
        # Split the actor's bodypart string into a list
        actor_bps = touch_row['bodypart_actor'].split()
        # Split the recipient's bodypart string into a list
        recipient_bps = touch_row['bodypart_recipient'].split()

        # Mark the actor body parts as 1
        for bp in actor_bps:
            col_name = f"{bp}_actor"
            if col_name in bodypart_encoding:
                bodypart_encoding[col_name] = 1

        # Mark the recipient body parts as 1
        for bp in recipient_bps:
            col_name = f"{bp}_recipient"
            if col_name in bodypart_encoding:
                bodypart_encoding[col_name] = 1

    # -- Combine the emotion data with the one-hot-encoded body parts
    aligned_rows.append({
        'offset':    current_offset,
        'start':     emotion_row.get('start'),
        'end':       emotion_row.get('end'),
        'character': emotion_row.get('character'),
        'arousal':   emotion_row.get('arousal'),
        'valence':   emotion_row.get('valence'),
        'direction': emotion_row.get('direction'),
        'emotion':   emotion_row.get('emotion'),
        'oncue':     emotion_row.get('oncue'),
        'offcue':    emotion_row.get('offcue'),
        **bodypart_encoding
    })

# -- Convert to DataFrame
emotion_df = pd.DataFrame(aligned_rows)


-- 4. Convert to DataFrame --

In [47]:
emotion_df = pd.DataFrame(aligned_rows)

# Display results
print(list(emotion_df.columns))
print(emotion_df.head())
row = emotion_df[emotion_df['offset'] == 1538]

if row.empty:
    print("No row found for offset 1538.")
else:
    row_dict = row.to_dict(orient="records")[0]  # Convert row to dictionary
    for key, value in row_dict.items():
        print(f"{key}: {value}")

print(f"Number of rows in emotion CSV: {emotion_df.shape[0]}")
print(f"Number of rows in touch CSV: {emotion_df.shape[0]}")


['offset', 'start', 'end', 'character', 'arousal', 'valence', 'direction', 'emotion', 'oncue', 'offcue', 'ABDOMEN_actor', 'ARM_actor', 'BACK_actor', 'BUTTOCKS_actor', 'CHEST_actor', 'FACE_actor', 'FOOT_actor', 'HAND_actor', 'HEAD_actor', 'HIP_actor', 'LAP_actor', 'LEG_actor', 'NAPE_actor', 'NECK_actor', 'SHOULDER_actor', 'ABDOMEN_recipient', 'ARM_recipient', 'BACK_recipient', 'BUTTOCKS_recipient', 'CHEST_recipient', 'FACE_recipient', 'FOOT_recipient', 'HAND_recipient', 'HEAD_recipient', 'HIP_recipient', 'LAP_recipient', 'LEG_recipient', 'NAPE_recipient', 'NECK_recipient', 'SHOULDER_recipient']
   offset  start  end character arousal valence direction emotion oncue  \
0       0    NaN  NaN       NaN     NaN     NaN       NaN    NONE   NaN   
1       2    NaN  NaN       NaN     NaN     NaN       NaN    NONE   NaN   
2       4    NaN  NaN       NaN     NaN     NaN       NaN    NONE   NaN   
3       6    NaN  NaN       NaN     NaN     NaN       NaN    NONE   NaN   
4       8    NaN  NaN   

-- 5. Export Dataframe --

In [48]:
output_path = "/Users/joshualunger/DeepEmotion/data/resampled_annotations/emotions_resampled.tsv"

# Export to TSV (preserving tab separator)
emotion_df.to_csv(output_path, sep='\t', index=False)

print(f"Emotion DataFrame saved to: {output_path}")

Emotion DataFrame saved to: /Users/joshualunger/DeepEmotion/data/resampled_annotations/emotions_resampled.tsv
